In [1]:
import jieba
import pandas as pd
import tensorflow as tf
from collections import Counter
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
path = r'C:\Users\Thinkpad\Desktop\源码\nlp基础\cnews'
train_data = pd.read_csv(path + r'\cnews.train.txt', names=['title', 'content'], sep='\t',engine='python',encoding='UTF-8')  # (50000, 2)
test_data = pd.read_csv(path + r'\cnews.test.txt', names=['title', 'content'], sep='\t',engine='python',encoding='UTF-8')  # (10000, 2)
val_data = pd.read_csv(path + r'\cnews.val.txt', names=['title', 'content'], sep='\t',engine='python',encoding='UTF-8')  # (5000, 2)


train_data = train_data.head(50)
test_data = test_data.head(50)
val_data = val_data.head(50)

In [6]:
train_data

,title,content
0,体育,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...
1,体育,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...
2,体育,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...
3,体育,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...
4,体育,揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道 谢亚龙已经被公安...
5,体育,阿的江：八一需重新定位 我们有机会但该进的没进新浪体育讯12月24日，回到主场的北京金隅迎战...
6,体育,姚明未来次节出战成疑 火箭高层称将改变用姚战略新浪体育讯北京时间11月5日消息，休斯敦当地6...
7,体育,姚明：我来承担一切 四连败巨人宣言酷似当年麦蒂新浪体育讯北京时间11月5日消息，据《休斯敦纪...
8,体育,姚麦均无胜殊途同归 活塞酝酿交易火箭不如插一脚新浪体育讯火箭和活塞，是全联盟仅存的两支没有胜...
9,体育,布雷克替补席成功接棒法玛 湖人板凳后卫越打越好新浪体育讯记者戴高乐报道 上赛季，湖人经常在...


In [5]:
def word_list(data):
    all_word = []
    for word in data['content']:
        all_word.extend(word)
    return all_word
word_list(train_data)

['马',
 '晓',
 '旭',
 '意',
 '外',
 '受',
 '伤',
 '让',
 '国',
 '奥',
 '警',
 '惕',
 ' ',
 '无',
 '奈',
 '大',
 '雨',
 '格',
 '外',
 '青',
 '睐',
 '殷',
 '家',
 '军',
 '记',
 '者',
 '傅',
 '亚',
 '雨',
 '沈',
 '阳',
 '报',
 '道',
 ' ',
 '来',
 '到',
 '沈',
 '阳',
 '，',
 '国',
 '奥',
 '队',
 '依',
 '然',
 '没',
 '有',
 '摆',
 '脱',
 '雨',
 '水',
 '的',
 '困',
 '扰',
 '。',
 '7',
 '月',
 '3',
 '1',
 '日',
 '下',
 '午',
 '6',
 '点',
 '，',
 '国',
 '奥',
 '队',
 '的',
 '日',
 '常',
 '训',
 '练',
 '再',
 '度',
 '受',
 '到',
 '大',
 '雨',
 '的',
 '干',
 '扰',
 '，',
 '无',
 '奈',
 '之',
 '下',
 '队',
 '员',
 '们',
 '只',
 '慢',
 '跑',
 '了',
 '2',
 '5',
 '分',
 '钟',
 '就',
 '草',
 '草',
 '收',
 '场',
 '。',
 '3',
 '1',
 '日',
 '上',
 '午',
 '1',
 '0',
 '点',
 '，',
 '国',
 '奥',
 '队',
 '在',
 '奥',
 '体',
 '中',
 '心',
 '外',
 '场',
 '训',
 '练',
 '的',
 '时',
 '候',
 '，',
 '天',
 '就',
 '是',
 '阴',
 '沉',
 '沉',
 '的',
 '，',
 '气',
 '象',
 '预',
 '报',
 '显',
 '示',
 '当',
 '天',
 '下',
 '午',
 '沈',
 '阳',
 '就',
 '有',
 '大',
 '雨',
 '，',
 '但',
 '幸',
 '好',
 '队',
 '伍',
 '上',
 '午',
 '的',
 '训',
 '练',
 '并',
 '没',
 '有',
 '受'

In [7]:
len(word_list(train_data))

53025

In [25]:
len(train_data.iloc[0,1])

746

In [14]:
def feature(train_data, test_data, val_data):
    content = pd.concat([train_data['content'], test_data['content'], val_data['content']], ignore_index=True)
    count_vec = CountVectorizer(max_features=300, min_df=2)
    count_vec.fit_transform(content)
    train_fea = count_vec.transform(train_data['content']).toarray()
    test_fea = count_vec.transform(test_data['content']).toarray()
    val_fea = count_vec.transform(val_data['content']).toarray()
    #model = Word2Vec(content, size=100, min_count=1, window=10, iter=10)
    #train_fea = train_data['content'].apply(lambda x: model[x])
    #test_fea = test_data['content'].apply(lambda x: model[x])
    #val_fea = val_data['content'].apply(lambda x: model[x])
    return train_fea, test_fea, val_fea,content

train_fea, test_fea, val_fea ,content= feature(train_data, test_data, val_data)

In [15]:
train_fea.shape

(50, 300)

In [16]:
content

0      马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...
1      商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...
2      冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...
3      辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...
4      揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道  谢亚龙已经被公安...
5      阿的江：八一需重新定位 我们有机会但该进的没进新浪体育讯12月24日，回到主场的北京金隅迎战...
6      姚明未来次节出战成疑 火箭高层称将改变用姚战略新浪体育讯北京时间11月5日消息，休斯敦当地6...
7      姚明：我来承担一切 四连败巨人宣言酷似当年麦蒂新浪体育讯北京时间11月5日消息，据《休斯敦纪...
8      姚麦均无胜殊途同归 活塞酝酿交易火箭不如插一脚新浪体育讯火箭和活塞，是全联盟仅存的两支没有胜...
9      布雷克替补席成功接棒法玛 湖人板凳后卫越打越好新浪体育讯记者戴高乐报道  上赛季，湖人经常在...
10     希伯杜防守体系初见成效 芝加哥铁牛阵已显露真容新浪体育讯记者邱星报道  汤姆-希伯杜，这个名...
11     希伯特由无名小卒变身内线轴心 黑版姚明脱胎换骨新浪体育讯记者高嵩报道  姚明还在寻找比赛的感...
12     投篮打铁防守上当！ 雷霆老三续约遭白眼纯属自找新浪体育讯北京时间11月5日，开拓者主场迎来了...
13     普里兹比拉恢复训练 内线板凳匪徒最迟本月底复出 新浪体育讯记者周超报道  最多23天，普里兹...
14     最强青年军对决雷霆先拔头筹 西北区谁才是老大？ 新浪体育讯在今天的比赛中，俄克拉荷马雷霆在加...
15     末节致命失误马丁却不困惑 称火箭仍需找关键进攻点新浪体育讯北京时间11月5日(休斯敦时间11...
16     阿的江：打北京队胜负五五开 对手一人令我很忌惮本报北京电 新赛季前五轮收获4胜1负的阿的江理...
17     杜兰特28分雷霆避三连败 开拓者加时低迷主场首败